# Resnet

### 1. 介绍
Resnet是2015年提出的残差神经网络结构，有效解决了深层神经网络效果不佳的问题。

论文链接：https://arxiv.org/pdf/1512.03385

### 2.创新点

#### 残差连接
假设x是输入，经过神经网络的前向传播后，得到输出h(x)。令神经网络所表示的函数为f，此时有f(x)=h(x)。

残差连接核心思想是：不再让网络直接学习h(x)，而是令h(x)=f(x)+x，即输出为神经网络函数的输出与输入之和，此时f(x)=h(x)-x，神经网络只需要
学习最终输出与输入之间的残差。这种方法有效解决了网络层越加越深引起的性能退化问题。论文指出，这种效果不是过拟合所引起，因为不仅测试误差增加，训练误差同样增加。

#### BN(相比于Alexnet)
Alexnet使用了LRN，即局部相应归一化。

Resnet使用的是Batch Normalization(BN)，即对当前的输入数据进行归一化处理，强制转换为均值为0，方差为1的标准分布。
相比于LRN，该方法能更有效地抑制过拟合，并可以让神经网络更专注地学习数据本身，而不是适应分布变化。

### 3.网络架构
Resnet引入了残差块与维度匹配，残差块是多层网络的结合体，Resnet中大量使用了结构相似的残差块。

维度匹配让网络能够连接输入与输出，想要令h(x)=f(x)+x，需要让f(x)与x的形状(维度)相同，当形状不同时，
原始的恒等映射x会被替换为1x1卷积层的投影映射，该卷积实现了下采样，令其形状相等。

下采样作用：降低特征图的高度与宽度（比如56x56降到28x28）；通道升维，例如64通道变成128通道。

*Pytorch实现如下：*

In [ ]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
    
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out
    
class Resnet18(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7,
                               stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(64, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        self.layer4 = self._make_layer(256, 512, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
    
    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        downsample = None
        
        # 当我们需要下采样时（通常是每个阶段的第一个残差块）stride != 1
        # 如果通道数不同就下采样，目的是为了便于残差连接，残差连接的输入和输出所需通道数相同。
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
        layers = []
        # 只有第一个块可能需要下采样
        layers.append(BasicBlock(in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x